# Matrix and Vector operation

Here, I will provide the basic operations for Eigen `Matrix` and `Vector`. The arithmetic is based upon C++ operator overloads such as `+`, `-` and `*`. Moreover, proper operations for linear algebra like `dot()`, `cross()`, etc are provided. For `Matrix` class, only operation which makes sense in Linear Algebra are allowed. For example, the sum of a scalar with a vector is not allowed. To perform general operations in a coefficient-wise manner, Eigen provides the `Array` class, which can be seem as a general-purpose arrays. This class will be explored in another notebook.

As in previous notebooks, the material presented here is highly based on the [Eigen's official documentation](https://eigen.tuxfamily.org/dox/group__TutorialMatrixArithmetic.html), except for few modifications and adaptations.